In [1]:
from brightness import dino

# Set constants here
W = 0.9
A = 1
B = 1
C = 1
D = 1
SCALE_NORMALIZED = True

def file_to_brightness(file_path, resize_width=None, resize_height=None):
    image = dino.read_image(file_path, resize_width=resize_width, resize_height=resize_height)
    X, Y, Z = dino.rgb_to_xyz(image)
    L, _, _ = dino.xyz_to_lxy(X, Y, Z)
    return dino.dn_brightness_model(
        L,
        min_scale=1,
        w=W,
        a=A,
        b=B,
        c=C,
        d=D,
        scale_normalized_constants=SCALE_NORMALIZED,
    )

In [2]:
import json
from pathlib import Path

import numpy as np
from tqdm.auto import tqdm

def MSE(img1, img2):
    assert img1.shape == img2.shape, f"{img1.shape} != {img2.shape}"
    return np.mean((img1 - img2) ** 2)

tonemapper_brightness_losses = {}
for image_name, (width, height) in [
    ("indoor", (600, 450)),
    ("outdoor", (600, 900)),
    ("night", (800, 533)),
]:
    directory = Path("cadik/tonemapped_images") / image_name
    hdr_brightness = None
    tonemapper_losses = {}

    print(f"Calculating brightness response for {image_name} HDR image")
    for file in directory.iterdir():
        if file.suffix == ".hdr":
            hdr_brightness = file_to_brightness(str(file), resize_width=width, resize_height=height)
            break
    print(f"Calculating brightness response for {image_name} tonemapped images")
    for file in tqdm([f for f in directory.iterdir() if f.suffix == ".png"], desc=image_name):
        tonemapped_brightness = file_to_brightness(str(file), resize_width=width, resize_height=height)
        tonemapper_losses[file.stem.lower()] = MSE(tonemapped_brightness, hdr_brightness)
    tonemapper_brightness_losses[image_name] = tonemapper_losses

print(json.dumps(tonemapper_brightness_losses, indent=2))

Calculating brightness response for indoor HDR image
Calculating brightness response for indoor tonemapped images


indoor:   0%|          | 0/14 [00:00<?, ?it/s]

Calculating brightness response for outdoor HDR image
Calculating brightness response for outdoor tonemapped images


outdoor:   0%|          | 0/14 [00:00<?, ?it/s]

Calculating brightness response for night HDR image
Calculating brightness response for night tonemapped images


night:   0%|          | 0/14 [00:00<?, ?it/s]

{
  "indoor": {
    "ashikhmin02": 0.10563428351082774,
    "chiu93": 0.1971990017861757,
    "choudhury03": 0.10219508079102245,
    "clip": 0.04154162973980786,
    "drago03": 0.06732608658761027,
    "durand02": 0.011478111996178928,
    "fattal02": 0.04995707262772405,
    "lcis99": 0.08330986564306792,
    "pattanaik02": 0.11292733663626023,
    "reinhard02": 0.042146602837662105,
    "schlick94": 0.012432439208294881,
    "tumblin99": 0.03582096496236,
    "ward94": 0.019821624943728373,
    "ward97": 0.04248335541832223
  },
  "outdoor": {
    "ashikhmin02": 0.06962537315643558,
    "chiu93": 0.16824147293226002,
    "choudhury03": 0.06276596457990437,
    "clip": 0.01815627345674986,
    "drago03": 0.03197469811894357,
    "durand02": 0.00234371213670279,
    "fattal02": 0.08596513789799691,
    "lcis99": 0.05198362982334359,
    "pattanaik02": 0.11160359996715354,
    "reinhard02": 0.017218294842410345,
    "schlick94": 0.011521334319357797,
    "tumblin99": 0.0141681124384660

In [3]:
import pandas as pd

ranking_df = pd.read_csv("cadik/ranking_evaluation.csv", header=[0,1])
rating_df = pd.read_csv("cadik/rating_evaluation.csv", header=[0,1])

In [4]:
tm_ranking_dfs = {}

for image_name in ["indoor", "outdoor", "night"]:
    print(image_name)

    headers = ["dino"]
    # print("tone mappers ranked by brightness model:")
    dino = []
    # Ascending order, lower loss is better
    for i, (tm, _) in enumerate(sorted(tonemapper_brightness_losses[image_name].items(), key=lambda x: x[1])):
        # print(f"{i+1}. {tm}")
        dino.append(tm)

    cols = [dino]
    for df, name in [(ranking_df, "ranking"), (rating_df, "rating")]:
        for header in df[image_name].columns[1:]:
            # print(f"tone mappers ranked by Cadik {header} {name}:")
            ranking = []
            # Descending order, higher ranking is better
            for i, tm in enumerate(df.sort_values((image_name, header), ascending=False)[image_name]["tm"].to_list()):
                # print(f"{i+1}. {tm}")
                ranking.append(tm)
            cols.append(ranking)
            headers.append(f"{header}_{name}")

    tm_ranking_df = pd.DataFrame({h: col for h, col in zip(headers, cols, strict=True)})
    tm_ranking_df.to_csv(f"cadik/{image_name}_tm_ranking.csv")
    tm_ranking_dfs[image_name] = tm_ranking_df

indoor
outdoor
night


In [5]:
print(tm_ranking_dfs["indoor"])

           dino brightness_ranking contrast_ranking details_ranking  \
0      durand02          tumblin99       reinhard02     pattanaik02   
1     schlick94        pattanaik02           ward97      reinhard02   
2        ward94            drago03          drago03      ashikmin02   
3     tumblin99         reinhard02        tumblin99          ward97   
4          clip               clip      pattanaik02            clip   
5    reinhard02             ward97           ward94       tumblin99   
6        ward97           durand02       ashikmin02        fattal02   
7      fattal02         ashikmin02             clip     choudhury03   
8       drago03             ward94        schlick94         drago03   
9        lcis99        choudhury03           lcis99        durand02   
10  choudhury03             lcis99      choudhury03       schlick94   
11  ashikhmin02          schlick94         fattal02          lcis99   
12  pattanaik02           fattal02         durand02          ward94   
13    